# STHSL End-to-End Runner — Baseline / Option1 / Option2 (NYC + CHI)

1. **Path verification**
2. Train NYC (baseline)
3. Test NYC (baseline) — `metrics_generation.py`
4. Extract + analyze NYC (baseline) — `metrics_extraction.py` + `analyse_metrics.py`
5. Repeat 2–4 for NYC (option1)
6. Repeat 2–4 for NYC (option2)
7. Repeat 2–6 for CHI


## 1) Path verification + shared helpers

In [ ]:

from pathlib import Path
import os, shlex, subprocess

# ---- EDIT ONLY IF NEEDED ----
REPO_ROOT = Path(".").resolve()
PYTHON = os.environ.get("PYTHON", "python")

TRAIN = REPO_ROOT / "train.py"
METRICS_GEN = REPO_ROOT / "metrics_generation.py"
METRICS_EXTRACT = REPO_ROOT / "metrics_extraction.py"
ANALYSE = REPO_ROOT / "analyse_metrics.py"

SAVE_DIR = REPO_ROOT / "Save"                 # training outputs
LOGS_DIR = REPO_ROOT / "Logs"                 # where you move checkpoints
TESTS_DIR = REPO_ROOT / "tests_validation"    # metrics_generation out-dir root

def must_exist(p: Path):
    if not p.exists():
        raise FileNotFoundError(f"Missing: {p}")
    print("OK:", p)

print("REPO_ROOT:", REPO_ROOT)
must_exist(TRAIN)
must_exist(METRICS_GEN)
must_exist(METRICS_EXTRACT)
must_exist(ANALYSE)

# Ensure dirs exist
SAVE_DIR.mkdir(exist_ok=True)
LOGS_DIR.mkdir(exist_ok=True)
TESTS_DIR.mkdir(exist_ok=True)

def run(cmd, cwd=REPO_ROOT, timeout=None):
    """Run a command (list[str] or str). Prints the command and last output."""
    if isinstance(cmd, str):
        printable = cmd
        shell = True
    else:
        printable = " ".join(shlex.quote(str(x)) for x in cmd)
        shell = False
    print("\\n$ " + printable)
    p = subprocess.run(
        cmd,
        cwd=str(cwd),
        shell=shell,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        encoding="utf-8",   # <-- forces decoding as UTF-8
        errors="replace",   # <-- prevents crash on bad bytes
        timeout=timeout
    )
    out = p.stdout or ""
    print(out[-4000:])
    if p.returncode != 0:
        raise RuntimeError(f"Command failed (rc={p.returncode}). See output above.")
    return out

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)
    return p

print("\nReady.")


## Conventions

- `--use_ode_option`: `baseline`, `option1`, `option2`
- Dataset values: `NYC`, `CHI`
- Test output: `tests_validation/<arch>/<dataset>/`
- Logs: `Logs/<arch>/<dataset>/`


In [ ]:

def logs_dir(arch: str, ds: str) -> Path:
    return LOGS_DIR / arch / ds

def tests_dir(arch: str, ds: str) -> Path:
    return TESTS_DIR / arch / ds

def metrics_pivot_csv(arch: str, ds: str) -> Path:
    return tests_dir(arch, ds) / "metrics_all_epochs_pivot.csv"


## Helper: exact pipeline steps (train → test → extract+analyze)

In [ ]:
import shutil
import glob
from pathlib import Path

def step_pretrain_cleanup(ds: str, arch: str, clean_logs=True, clean_tests=True, clean_save=True):
    # Always safest: clean Save/<ds> before training to avoid mixing epochs
    if clean_save:
        shutil.rmtree(SAVE_DIR / ds, ignore_errors=True)

    if clean_logs:
        shutil.rmtree(LOGS_DIR / arch / ds, ignore_errors=True)

    if clean_tests:
        shutil.rmtree(TESTS_DIR / arch / ds, ignore_errors=True)

    print(f"Pre-train cleanup: ds={ds}, arch={arch} (save={clean_save}, logs={clean_logs}, tests={clean_tests})")

def step_train(ds: str, arch: str):
    # baseline: python train.py --data NYC
    # option1:  python train.py --data NYC --use_ode_option option1
    # option2:  python train.py --data NYC --use_ode_option option2
    cmd = [PYTHON, str(TRAIN), "--data", ds]
    if arch in ("option1", "option2"):
        cmd += ["--use_ode_option", arch]
    elif arch == "baseline":
        cmd += ["--use_ode_option", "baseline"]
        pass
    else:
        raise ValueError(f"Unknown arch: {arch}")
    run(cmd)

def step_move_checkpoints(
    ds: str,
    arch: str,
    on_conflict: str = "prompt",  # "abort" | "overwrite" | "skip" | "prompt"
):
    """
    Move checkpoints from Save/<ds>/_epoch_* to Logs/<arch>/<ds>/.

    on_conflict behavior:
      - "abort"     : stop if destination files exist (default, safest)
      - "overwrite" : overwrite existing files
      - "skip"      : skip files that already exist
      - "prompt"    : ask user interactively (not for batch runs)
    """
    src_dir = SAVE_DIR / ds
    dst_dir = ensure_dir(logs_dir(arch, ds))

    pattern = str(src_dir / "_epoch_*")
    src_files = sorted(glob.glob(pattern))

    if not src_files:
        raise RuntimeError(f"No checkpoints found matching {pattern}")

    # Detect conflicts
    conflicts = []
    for f in src_files:
        if (dst_dir / Path(f).name).exists():
            conflicts.append(Path(f).name)

    # Resolve conflict policy
    if conflicts:
        if on_conflict == "abort":
            raise RuntimeError(
                f"Checkpoint conflicts detected in {dst_dir}. "
                f"Refusing to overwrite: {conflicts}"
            )

        if on_conflict == "prompt":
            print("\n Existing checkpoint files detected:")
            for c in conflicts:
                print("   -", c)

            print("\nChoose an action:")
            print("  [o] overwrite existing files")
            print("  [s] skip existing files")
            print("  [a] abort")

            choice = input("Your choice [o/s/a]: ").strip().lower()
            if choice == "o":
                on_conflict = "overwrite"
            elif choice == "s":
                on_conflict = "skip"
            else:
                print("Aborted by user.")
                return

        if on_conflict not in ("overwrite", "skip"):
            raise ValueError(f"Invalid on_conflict value: {on_conflict}")

    moved, skipped = 0, 0

    for f in src_files:
        src = Path(f)
        dst = dst_dir / src.name

        if dst.exists():
            if on_conflict == "skip":
                skipped += 1
                continue
            elif on_conflict == "overwrite":
                dst.unlink()

        shutil.move(str(src), str(dst))
        moved += 1

    print(f"\n Checkpoint move completed ({ds} / {arch})")
    print(f"   Moved   : {moved}")
    print(f"   Skipped : {skipped}")

def step_test_metrics_generation(ds: str, arch: str):
    # example command: python metrics_generation.py --data NYC --use_ode_option baseline --out-dir tests_validation/baseline/NYC
    out_dir = ensure_dir(tests_dir(arch, ds))
    cmd = [PYTHON, str(METRICS_GEN), "--data", ds, "--use_ode_option", arch, "--save-root", LOGS_DIR/arch, "--out-dir", str(out_dir)]
    run(cmd)

def step_metrics_extraction(ds: str, arch: str):
    # example command: python metrics_extraction.py tests_validation/baseline/NYC/
    out_dir = tests_dir(arch, ds)
    cmd = [PYTHON, str(METRICS_EXTRACT), str(out_dir)]
    run(cmd)

def step_analyse(ds: str, arch: str):
    # example command: python analyse_metrics.py tests_validation/baseline/NYC/metrics_all_epochs_pivot.csv
    pivot = metrics_pivot_csv(arch, ds)
    cmd = [PYTHON, str(ANALYSE), str(pivot)]
    run(cmd)

def run_full(ds: str, arch: str, do_move_ckpt: bool = True, auto_cleanup: bool = True):
    print(f"\\n=== RUN FULL: ds={ds}, arch={arch} ===")
    if auto_cleanup:
        step_pretrain_cleanup(ds, arch, clean_logs=True, clean_tests=True, clean_save=True)
    step_train(ds, arch)
    if do_move_ckpt:
        step_move_checkpoints(ds, arch, on_conflict="overwrite")
    step_test_metrics_generation(ds, arch)
    step_metrics_extraction(ds, arch)
    step_analyse(ds, arch)
    print(f"=== DONE: ds={ds}, arch={arch} ===\\n")


# Manual Hyper parameter tuning

In [ ]:
# !python train.py --data NYC --use_ode_option option1 --hyperNum 256

In [ ]:
# !python train.py --data NYC --use_ode_option option2 --hyperNum 256
# !python train.py --data CHI --use_ode_option option1 --hyperNum 256
# !python train.py --data CHI --use_ode_option option2 --hyperNum 256

In [ ]:
# !python train.py --data NYC --use_ode_option option1 --hyperNum 64
# step_move_checkpoints("NYC", "option1", on_conflict="overwrite")
# !python train.py --data NYC --use_ode_option option2 --hyperNum 64
# step_move_checkpoints("NYC", "option2", on_conflict="overwrite")
# !python train.py --data CHI --use_ode_option option1 --hyperNum 64
# step_move_checkpoints("CHI", "option1", on_conflict="overwrite")
# !python train.py --data CHI --use_ode_option option2 --hyperNum 64
# step_move_checkpoints("CHI", "option2", on_conflict="overwrite")

In [ ]:
step_train("NYC", "baseline")
step_move_checkpoints("NYC", "baseline", on_conflict="overwrite")
step_test_metrics_generation("NYC", "baseline")
step_metrics_extraction("NYC", "baseline")
step_analyse("NYC", "baseline")

In [ ]:
step_train("NYC", "option1")
step_move_checkpoints("NYC", "option1", on_conflict="overwrite")
step_test_metrics_generation("NYC", "option1")
step_metrics_extraction("NYC", "option1")
step_analyse("NYC", "option1")

In [ ]:
step_train("NYC", "option2")
step_move_checkpoints("NYC", "option2", on_conflict="overwrite")
step_test_metrics_generation("NYC", "option2")
step_metrics_extraction("NYC", "option2")
step_analyse("NYC", "option2")

In [ ]:
# test cell

# NYC dataset

# NYC — baseline

## Training (NYC, baseline)

In [ ]:
step_train("NYC", "baseline")

## Move checkpoints Save → Logs (NYC, baseline)

In [ ]:
step_move_checkpoints("NYC", "baseline", on_conflict="overwrite")

## Testing / metric generation (NYC, baseline)

In [ ]:
step_test_metrics_generation("NYC", "baseline")

## Metric extraction + analysis (NYC, baseline)

In [ ]:
step_metrics_extraction("NYC", "baseline")
step_analyse("NYC", "baseline")

# NYC — option1

## Training (NYC, option1)

In [ ]:
step_train("NYC", "option1")

## Move checkpoints Save → Logs (NYC, option1)

In [ ]:
step_move_checkpoints("NYC", "option1", on_conflict="overwrite")

## Testing / metric generation (NYC, option1)

In [ ]:
step_test_metrics_generation("NYC", "option1")

## Metric extraction + analysis (NYC, option1)

In [ ]:
step_metrics_extraction("NYC", "option1")
step_analyse("NYC", "option1")

# NYC — option2

## Training (NYC, option2)

In [ ]:
step_train("NYC", "option2")

## Move checkpoints Save → Logs (NYC, option2)

In [ ]:
step_move_checkpoints("NYC", "option2", on_conflict="overwrite")

## Testing / metric generation (NYC, option2)

In [ ]:
step_test_metrics_generation("NYC", "option2")

## Metric extraction + analysis (NYC, option2)

In [ ]:
step_metrics_extraction("NYC", "option2")
step_analyse("NYC", "option2")

# CHI dataset

# CHI — baseline

## Training (CHI, baseline)

In [ ]:
step_train("CHI", "baseline")

## Move checkpoints Save → Logs (CHI, baseline)

In [ ]:
step_move_checkpoints("CHI", "baseline", on_conflict="overwrite")


## Testing / metric generation (CHI, baseline)

In [ ]:
step_test_metrics_generation("CHI", "baseline")

## Metric extraction + analysis (CHI, baseline)

In [ ]:
step_metrics_extraction("CHI", "baseline")
step_analyse("CHI", "baseline")

# CHI — option1

## Training (CHI, option1)

In [ ]:
step_train("CHI", "option1")

## Move checkpoints Save → Logs (CHI, option1)

In [ ]:
step_move_checkpoints("CHI", "option1", on_conflict="overwrite")

## Testing / metric generation (CHI, option1)

In [ ]:
step_test_metrics_generation("CHI", "option1")

## Metric extraction + analysis (CHI, option1)

In [ ]:
step_metrics_extraction("CHI", "option1")
step_analyse("CHI", "option1")

# CHI — option2

## Training (CHI, option2)

In [ ]:
step_train("CHI", "option2")

## Move checkpoints Save → Logs (CHI, option2)

In [ ]:
step_move_checkpoints("CHI", "option2", on_conflict="overwrite")

## Testing / metric generation (CHI, option2)

In [ ]:
step_test_metrics_generation("CHI", "option2")

## Metric extraction + analysis (CHI, option2)

In [ ]:
step_metrics_extraction("CHI", "option2")
step_analyse("CHI", "option2")

## Night runs

In [ ]:
# Run all 3 architectures for NYC
# for arch in ["baseline", "option1", "option2"]:
#    run_full("NYC", arch, do_move_ckpt=True, auto_cleanup=True)

In [ ]:
# Run all 3 architectures for CHI
# for arch in ["baseline", "option1", "option2"]:
#    run_full("CHI", arch, do_move_ckpt=True, auto_cleanup=True)

In [ ]:
# Run everything (NYC + CHI) for baseline/option1/option2
# for ds in ["NYC", "CHI"]:
#    for arch in ["baseline", "option1", "option2"]:
#        run_full(ds, arch, do_move_ckpt=True, auto_cleanup=True)